In [8]:
import pandas as pd
import numpy as np
import os
import imp
import telephone_analysis
import srilm
import roark
import glob
import shutil
import scipy.stats
imp.reload(telephone_analysis)
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import kenlm

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [9]:
all_runs  = pd.read_csv('output/all_runs.csv')
# run these through each of the language models and merge the results
# make a word level table
# diff the words to see if they are not present in the next generation

In [160]:
all_runs.loc[all_runs.user != "0"].shape[0]

2999

In [162]:
len(np.unique(all_runs.loc[all_runs.user != "0"].user))

266

In [11]:
all_runs.columns

Index(['Unnamed: 0', 'chain', 'character_levdau', 'check_time', 'condition',
       'flag_type', 'gold_candidate_transcription',
       'gold_comparison_transcription', 'gold_dist', 'length_accept', 'reason',
       'run', 'stimulus', 'stimulus_id', 'subject_id', 'target_length',
       'upload_time', 'upstream_pointer', 'upstream_subject_id', 'user',
       'user_candidate_transcription', 'user_comparison_transcription',
       'user_dist', 'user_short', 'word_distances', 'unique_chain_identifier',
       'global_chain', 'BNC_KNN_trigramProb', 'BNC_KNN_unigramProb',
       'WSJ_gt_unigramProb', 'WSJ_gt_trigramProb', 'WSJ_gt_5gramProb',
       'WSJ_Roark_Negative.Log.Probability', 'biglm_probability',
       'kenlm_probability', 'bllip_probability', 'bllip_wsj_probability',
       'mikolov_wsj_probability', 'length_in_words',
       'normalized_biglm_probability',
       'normalized_WSJ_Roark_Negative.Log.Probability',
       'normalized_BNC_KNN_unigramProb', 'normalized_BNC_KNN_trigra

# Word-Level Language Modeling Results

In [5]:
# index in the LM corresponds to the response: user_candidate_transcription. 3192: last input

In [12]:
lm = {}

In [13]:
bnc_knn_lm = srilm.LM("LMs/BNC_merged.LM", lower=True)

In [14]:
lm['bnc_unigram'] =  [telephone_analysis.getSRILMprob(x, {}, bnc_knn_lm, mode='single_word', unigram=True) for x in all_runs['user_candidate_transcription']]

In [15]:
assert(len(lm['bnc_unigram']) == all_runs.shape[0])

In [16]:
lm['bnc_trigram'] =  [telephone_analysis.getSRILMprob(x, {}, bnc_knn_lm, mode='single_word', unigram = False) for x in all_runs['user_candidate_transcription']]

In [17]:
assert(len(lm['bnc_trigram']) == all_runs.shape[0])

In [18]:
lm['roark_scores'] = roark.parse([str(i) for i in all_runs['user_candidate_transcription']], 
    '/home/stephan/utils/incremental-top-down-parser',
                          numWorkers=24, mode='single_word')

Input contains 3193 sentences
Input contains 1982 unique sentences
Finished external parsing


/home/stephan/notebooks/telephone-analysis-public/roark.py:235: FutureWarning: read_table is deprecated, use read_csv instead.
  pfix_df = pd.read_table(StringIO('\n'.join(y)), sep=' ', header=None, names = ['prefix','word']+colnames)
/home/stephan/notebooks/telephone-analysis-public/lib/python3.6/site-packages/pandas/io/parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [19]:
#roark_scores.columns = ['WSJ_Roark_'+x.replace(' ','.') for x in roark_scores.columns]
assert len(lm['roark_scores']) == all_runs.shape[0]

In [20]:
lm['big_lm_scores'] = telephone_analysis.getBigLMscores(all_runs['user_candidate_transcription'], 'big_lm',
    'big_lm_cache', colname='BigLM_probability', lm_1b_dir='/home/stephan/python/lm_1b', mode='single_word')


Big LM call:
cd /home/stephan/python/lm_1b && source bin/activate && bazel-bin/lm_1b/lm_1b_eval --mode eval_sentences --pbtxt data/graph-2016-09-10.pbtxt --vocab_file data/vocab-2016-09-10.txt --eval_dir /home/stephan/notebooks/telephone-analysis-public/big_lm --ckpt 'data/ckpt-*'
127


In [21]:
assert(len(lm['big_lm_scores']) == all_runs.shape[0])

In [22]:
m = kenlm.Model('LMs/deepspeech_5gram.binary')
lm['kenlm_scores'] = all_runs['kenlm_probability'] = [telephone_analysis.getKenLMProb(x, m, mode='single_word') for x in all_runs['user_candidate_transcription']]

In [23]:
assert(len(lm['kenlm_scores']) == all_runs.shape[0])

In [25]:
# sanity check: all sentences in all models must have the same number of tokens in the dataframe
languageModelNames = ['bnc_unigram', 'bnc_trigram', 'roark_scores', 'big_lm_scores', 'kenlm_scores']
for i in range(all_runs.shape[0]):
    sentences = [lm[x][i] for x in languageModelNames]
    try:
        numWords = [x.shape[0] for x in sentences]
    except:
        print('Problem counting words')
        import pdb
        pdb.set_trace()
    if not np.allclose(numWords[1:len(numWords)], numWords[0]):
        print('Different number of words')
        import pdb
        pdb.set_trace()

In [26]:
sentence_store = []
for i in range(all_runs.shape[0]):
    # need to rename what is coming out of each language model
    word_store = [] 
    for languageModelName in languageModelNames:
        df = lm[languageModelName][i].copy()
        try:
            df.columns = [languageModelName+'_'+x for x in df.columns]        
        except:
            import pdb
            pdb.set_trace()
        word_store.append(df)
    lms_combined = pd.concat(word_store, axis =1)
    lms_combined['sCounter'] = range(lms_combined.shape[0])
    lms_combined['sentence_index'] = i
    sentence_store.append(lms_combined)
wdf = pd.concat(sentence_store)

In [27]:
wdf['word'] = wdf[u'bnc_unigram_word']

In [28]:
wdf.head()

,bnc_unigram_index,bnc_unigram_prob,bnc_unigram_word,bnc_trigram_index,bnc_trigram_prob,bnc_trigram_word,roark_scores_prefix,roark_scores_word,roark_scores_prefix.1,roark_scores_srprsl,...,big_lm_scores_Unnamed: 0,big_lm_scores_prob,big_lm_scores_word,kenlm_scores_prob,kenlm_scores_preceding,kenlm_scores_unk,kenlm_scores_words,sCounter,sentence_index,word
0,0.0,-3.310068,each,0.0,-2.994068,each,pfix:1,each,6.566,2.851578,...,0.0,-3.202744,Each,-3.516777,2.0,False,each,0,0,each
1,1.0,-7.309903,nonfiction,1.0,-8.412464,nonfiction,pfix:2,nonfiction,18.290,5.091669,...,1.0,-6.637383,nonfiction,-5.568481,1.0,False,nonfiction,1,0,nonfiction
2,2.0,-3.631093,book,2.0,-3.756868,book,pfix:3,book,28.519,4.442398,...,2.0,-0.854058,book,-1.820345,2.0,False,book,2,0,book
3,3.0,-2.600407,has,3.0,-2.253614,has,pfix:4,has,32.402,1.686800,...,3.0,-1.230795,has,-2.480787,2.0,False,has,3,0,has
4,4.0,-1.681165,a,4.0,-1.317157,a,pfix:5,a,35.256,1.239042,...,4.0,-0.638269,a,-1.162695,3.0,False,a,4,0,a


In [ ]:
# Each record is a 'produced' word (user_candidate_transcription) 

# Identifying Deleted and Changed Words

In [21]:
# for generation n, get the indices of all words that have changed in n+1
# have changed: no longer appear? doesn't handle transpositions
# borrowed the function from the old version of telephone

In [33]:
all_runs.loc[all_runs.user == "0"].head(5)

,Unnamed: 0,chain,character_levdau,check_time,condition,flag_type,gold_candidate_transcription,gold_comparison_transcription,gold_dist,length_accept,...,initial_WSJ_gt_unigramProb_rank,initial_WSJ_gt_unigramProb_quartile,initial_WSJ_gt_trigramProb,initial_WSJ_gt_trigramProb_rank,initial_WSJ_gt_trigramProb_quartile,initial_WSJ_gt_5gramProb,initial_WSJ_gt_5gramProb_rank,initial_WSJ_gt_5gramProb_quartile,thread_id,chain_length
0,0,0,NaN,NaN,none,none,none,none,NaN,NaN,...,31,3.0,-25.893792,17,1.0,-25.893792,17,1.0,0_0,24
18,18,1,NaN,NaN,none,none,none,none,NaN,NaN,...,31,3.0,-25.893792,17,1.0,-25.893792,17,1.0,1_0,0
20,20,0,NaN,NaN,none,none,none,none,NaN,NaN,...,31,3.0,-25.893792,17,1.0,-25.893792,17,1.0,3_0,15
33,33,1,NaN,NaN,none,none,none,none,NaN,NaN,...,31,3.0,-25.893792,17,1.0,-25.893792,17,1.0,4_0,14
48,48,2,NaN,NaN,none,none,none,none,NaN,NaN,...,31,3.0,-25.893792,17,1.0,-25.893792,17,1.0,5_0,20


In [108]:
# take a 2-column subset of all_trials that we can put into R
input_output = all_runs[['gold_candidate_transcription','user_candidate_transcription','user',
                         'upstream_subject_id']]
#gold_candidate_transcription is what a participant heard
#user_candidate_transcription is what the participant produced

# remove the intitial sentences -- these are represnted as input for the first participant
%R -i input_output

/home/stephan/notebooks/telephone-analysis-public/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "gold_candidate_transcription". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  % (name, str(e)))
/home/stephan/notebooks/telephone-analysis-public/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "user_candidate_transcription". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  % (name, str(e)))
/home/stephan/notebooks/telephone-analysis-public/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error while trying to convert the column "user". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  % (name, str(e)))
/home/stephan/notebooks/telephone-analysis-public/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:63: UserWarning: Error

In [111]:
input_output

,gold_candidate_transcription,user_candidate_transcription,user,upstream_subject_id
0,none,each nonfiction book has a call number on its ...,0,none
1,each nonfiction book has a call number on its ...,each non fiction book has a call number in its...,8cf6535ea0ae4addb28f5f90a2b13a7d,0
2,each non fiction book has a call number in its...,each nonfiction book had a call number on its ...,edf8f705ab7241a191fa3ae24e381ebc,8cf6535ea0ae4addb28f5f90a2b13a7d
3,Each nonfiction book had a call number on its ...,each nonfiction book had a call number on its ...,5bcd571bb9b645bc92fe6a4a7d5cfb35,edf8f705ab7241a191fa3ae24e381ebc
4,each nonfiction book had a call number on its ...,each non fiction book had a call number on its...,d14ebf1fffae49d8b76323eb6c393120,5bcd571bb9b645bc92fe6a4a7d5cfb35
5,each non fiction book had a call number on its...,each nonfiction book had a call number on its ...,67497f6ffb7e47d795ea7b655ccdd0ca,d14ebf1fffae49d8b76323eb6c393120
6,each nonfiction book had a call number on its ...,each non fiction book had 12 numbers on its spine,dc02e8bc38234eed94e4e0cfbe083801,67497f6ffb7e47d795ea7b655ccdd0ca
7,Each non fiction book had 12 numbers on its spine,each nonfiction book had twelve numbers on its...,e1f5fb8bc21048cb8a50771b80eda39a,dc02e8bc38234eed94e4e0cfbe083801
8,Each nonfiction book had twelve numbers on its...,each non fiction book had twelve numbers on it...,53edaed0aee2422e96af6c0d3d636962,e1f5fb8bc21048cb8a50771b80eda39a
9,each non fiction book had twelve numbers on it...,each nonfiction book had twelve notches on its...,a575811edc0841b082724370f678d574,53edaed0aee2422e96af6c0d3d636962


In [114]:
%%R
source('getWordLevenshteinDistance.R')
print(paste(nrow(input_output), 'sentences'))
names(input_output) = c('input','output','output_subject','input_subject')  
input_output$input = tolower(as.character(input_output$input))
input_output$output = tolower(as.character(input_output$output))

computeEditTable = function(s,r,input_subject, output_subject){
    if (s == 'none'){
        # this is an initial sentence, return NA
        return(NA)
    } else {
        et = getReducedEditTable(s,r)
        # for python compatibility, use 0-indices
        et$sCounter = et$sCounter - 1
        et$rCounter = et$rCounter - 1
        et$input_subject = input_subject
        et$output_subject = output_subject
        return(et)
    }
}

editTables = mapply(computeEditTable, input_output$input, input_output$output,
                   input_output$input_subject, input_output$output_subject)

[1] "3193 sentences"


In [116]:
%%R
editTables[[2]] #this should correspond to wdf[,1]

   code      sWord   rWord sCounter rCounter
1     M       each    each        0        0
2     D nonfiction    <NA>        1       NA
3     I       <NA>     non       NA        1
4     I       <NA> fiction       NA        2
5     M       book    book        2        3
6     M        has     has        3        4
7     M          a       a        4        5
8     M       call    call        5        6
9     M     number  number        6        7
11    S         on      in        7        8
12    M        its     its        8        9
13    M      spine   spine        9       10
                                              sentence
1  each nonfiction book has a call number on its spine
2  each nonfiction book has a call number on its spine
3  each nonfiction book has a call number on its spine
4  each nonfiction book has a call number on its spine
5  each nonfiction book has a call number on its spine
6  each nonfiction book has a call number on its spine
7  each nonfiction book has a 

In [120]:
%%R
print(computeEditTable('this is a', 'this is a mouse', NULL, NULL))
print(computeEditTable('this is a mouse', 'this is a', NULL, NULL ))
print(computeEditTable('this is a mouse', 'this is a mouse', NULL, NULL))
print(computeEditTable('this is a mouse', 'this is a house', NULL, NULL))

  code sWord rWord sCounter rCounter  sentence        response sLeftSequence
1    M  this  this        0        0 this is a this is a mouse              
2    M    is    is        1        1 this is a this is a mouse              
3    M     a     a        2        2 this is a this is a mouse     this is a
4    I  <NA> mouse       NA        3 this is a this is a mouse              
  rLeftSequence sRightSequence rRightSequence
1                    this is a      this is a
2                                  is a mouse
3     this is a                              
4    is a mouse                              
  code sWord rWord sCounter rCounter        sentence  response sLeftSequence
1    M  this  this        0        0 this is a mouse this is a              
2    M    is    is        1        1 this is a mouse this is a              
3    M     a     a        2        2 this is a mouse this is a     this is a
4    D mouse  <NA>        3       NA this is a mouse this is a    is a mouse


In [133]:
%%R
getDSMwrtInput = function(sentence_index, editTables){
    et = editTables[[sentence_index]]
    if (is.na(et)){
        return(data.frame(sentence_index, sCounter=NA,code=NA, input_subject=NA, output_subject=NA)) # these are initial sentences
    } else{
        et = editTables[[sentence_index]]    
        et$sentence_index = sentence_index
        # here is where insertions are removed so that we can join back with wfds
        return(subset(et, !is.na(sCounter))[,c('sentence_index','sCounter','code','input_subject','output_subject')])
    }
}

DSMwrtInput = do.call('rbind', lapply(c(1:length(editTables)), function(i){
    dsm = getDSMwrtInput(i, editTables)  
    if (!is.na(dsm)){
        dsm$sentence_index = dsm$sentence_index - 2 
        # -1 because Python indexes from 0
        # another -1 to bring the index of edits into alignment with the languag model results     
    } 
    return(dsm)
}))   

#DSMwrtInput = subset(DSMwrtInput, !is.na(code))

In [134]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
DSMwrtInput = r['DSMwrtInput']
DSMwrtInput.iloc[0:15,]

,sentence_index,sCounter,code,input_subject,output_subject
1,-1.0,NaN,NA_character_,NA_character_,NA_character_
110,0.0,0.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
2,0.0,1.0,D,0,8cf6535ea0ae4addb28f5f90a2b13a7d
5,0.0,2.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
6,0.0,3.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
7,0.0,4.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
8,0.0,5.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
9,0.0,6.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
11,0.0,7.0,S,0,8cf6535ea0ae4addb28f5f90a2b13a7d
12,0.0,8.0,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d


In [135]:
initialSentenceIndices = np.array(DSMwrtInput[np.isnan(DSMwrtInput.sCounter)]['sentence_index'].tolist())
wdf_initRemoved = wdf[(~wdf['sentence_index'].isin(initialSentenceIndices)) & (wdf['bnc_unigram_word'] != '</s>')]
DSMwrtInput_nansRemoved = DSMwrtInput[~np.isnan(DSMwrtInput.sCounter)]

In [136]:
print(DSMwrtInput_nansRemoved.shape[0])
print(wdf_initRemoved.shape[0])
#assert(DSMwrtInput_nansRemoved.shape[0] == wdf_initRemoved.shape[0])

27290
30298


In [137]:
wdf_initRemoved[wdf_initRemoved.sentence_index == 3192]

,bnc_unigram_index,bnc_unigram_prob,bnc_unigram_word,bnc_trigram_index,bnc_trigram_prob,bnc_trigram_word,roark_scores_prefix,roark_scores_word,roark_scores_prefix.1,roark_scores_srprsl,...,big_lm_scores_Unnamed: 0,big_lm_scores_prob,big_lm_scores_word,kenlm_scores_prob,kenlm_scores_preceding,kenlm_scores_unk,kenlm_scores_words,sCounter,sentence_index,word
0,0.0,-3.081292,before,0.0,-3.037731,before,pfix:1,before,6.993,3.037021,...,0.0,-3.060103,Before,-3.105574,2.0,False,before,0,3192,before
1,1.0,-2.186153,you,1.0,-1.185921,you,pfix:2,you,13.198,2.694363,...,1.0,-2.458716,you,-1.241865,3.0,False,you,1,3192,you
2,2.0,-3.713525,leave,2.0,-1.507196,leave,pfix:3,leave,20.940,3.362308,...,2.0,-2.782638,leave,-1.473630,4.0,False,leave,2,3192,leave
3,3.0,-3.121925,make,3.0,-4.336845,make,pfix:4,make,30.850,4.303858,...,3.0,-4.497365,make,-4.662948,1.0,False,make,3,3192,make
4,4.0,-3.643197,sure,4.0,-1.549859,sure,pfix:5,sure,36.383,2.402951,...,4.0,-1.128034,sure,-1.491675,2.0,False,sure,4,3192,sure
5,5.0,-2.186153,you,5.0,-0.752118,you,pfix:6,you,43.568,3.120406,...,5.0,-0.309889,you,-0.993335,3.0,False,you,5,3192,you
6,6.0,-3.512815,change,6.0,-3.870051,change,pfix:7,change,54.143,4.593098,...,6.0,-3.559230,change,-3.912920,2.0,False,change,6,3192,change
7,7.0,-3.763360,everything,7.0,-3.264657,everything,pfix:8,everything,63.772,4.181822,...,7.0,-3.270689,everything,-3.290390,2.0,False,everything,7,3192,everything
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,-1.231726,</S>,NaN,NaN,NaN,NaN,9,3192,NaN


In [138]:
DSMwrtInput_nansRemoved[DSMwrtInput_nansRemoved.sentence_index == 3192] 

,sentence_index,sCounter,code,input_subject,output_subject


In [139]:
dsm_counts = DSMwrtInput_nansRemoved.groupby(['sentence_index']).sCounter.agg(np.size).reset_index()

dsm_counts.columns = ['sentence_index', 'dsm_count']
wdf_counts = wdf_initRemoved.groupby('sentence_index').sCounter.agg(np.size).reset_index()
wdf_counts.columns = ['sentence_index', 'wdf_count']
count_check = wdf_counts.merge(dsm_counts, how='outer')
count_check # differences in the counts?
count_check['difference'] = count_check.wdf_count - count_check.dsm_count
count_check.sort_values(by=['difference'], ascending=False)

,sentence_index,wdf_count,dsm_count,difference
0,0,11,10.0,1.0
2003,2141,11,10.0,1.0
1994,2131,10,9.0,1.0
1995,2132,10,9.0,1.0
1996,2133,10,9.0,1.0
1997,2134,10,9.0,1.0
1998,2135,10,9.0,1.0
1999,2136,11,10.0,1.0
2000,2138,11,10.0,1.0
2001,2139,11,10.0,1.0


In [140]:
# merge DSMwrtInput into the word data frame
wdfr = wdf_initRemoved.merge(DSMwrtInput_nansRemoved) #sCounter is NaN for input sentences

In [141]:
#where did all my happy data go?
wdfr.code.value_counts()

M    22482
D     3442
S     1366
Name: code, dtype: int64

In [142]:
wdfr

,bnc_unigram_index,bnc_unigram_prob,bnc_unigram_word,bnc_trigram_index,bnc_trigram_prob,bnc_trigram_word,roark_scores_prefix,roark_scores_word,roark_scores_prefix.1,roark_scores_srprsl,...,kenlm_scores_prob,kenlm_scores_preceding,kenlm_scores_unk,kenlm_scores_words,sCounter,sentence_index,word,code,input_subject,output_subject
0,0.0,-3.310068,each,0.0,-2.994068,each,pfix:1,each,6.566,2.851578,...,-3.516777,2.0,False,each,0,0,each,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
1,1.0,-7.309903,nonfiction,1.0,-8.412464,nonfiction,pfix:2,nonfiction,18.290,5.091669,...,-5.568481,1.0,False,nonfiction,1,0,nonfiction,D,0,8cf6535ea0ae4addb28f5f90a2b13a7d
2,2.0,-3.631093,book,2.0,-3.756868,book,pfix:3,book,28.519,4.442398,...,-1.820345,2.0,False,book,2,0,book,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
3,3.0,-2.600407,has,3.0,-2.253614,has,pfix:4,has,32.402,1.686800,...,-2.480787,2.0,False,has,3,0,has,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
4,4.0,-1.681165,a,4.0,-1.317157,a,pfix:5,a,35.256,1.239042,...,-1.162695,3.0,False,a,4,0,a,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
5,5.0,-3.737589,call,5.0,-3.763689,call,pfix:6,call,44.218,3.892147,...,-3.580373,3.0,False,call,5,0,call,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
6,6.0,-3.321531,number,6.0,-3.708286,number,pfix:7,number,53.366,3.972926,...,-4.631904,1.0,False,number,6,0,number,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
7,7.0,-2.162270,on,7.0,-2.029635,on,pfix:8,on,58.319,2.151495,...,-1.878931,2.0,False,on,7,0,on,S,0,8cf6535ea0ae4addb28f5f90a2b13a7d
8,8.0,-2.804867,its,8.0,-2.394903,its,pfix:9,its,62.220,1.693748,...,-2.427654,2.0,False,its,8,0,its,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d
9,9.0,-5.034822,spine,9.0,-4.401074,spine,pfix:10,spine,77.301,6.549595,...,-6.263567,1.0,False,spine,9,0,spine,M,0,8cf6535ea0ae4addb28f5f90a2b13a7d


In [143]:
wdfr.columns

Index(['bnc_unigram_index', 'bnc_unigram_prob', 'bnc_unigram_word',
       'bnc_trigram_index', 'bnc_trigram_prob', 'bnc_trigram_word',
       'roark_scores_prefix', 'roark_scores_word', 'roark_scores_prefix.1',
       'roark_scores_srprsl', 'roark_scores_SynSp', 'roark_scores_LexSp',
       'roark_scores_ambig', 'roark_scores_open', 'roark_scores_rernk',
       'roark_scores_toprr', 'roark_scores_stps', 'big_lm_scores_Unnamed: 0',
       'big_lm_scores_prob', 'big_lm_scores_word', 'kenlm_scores_prob',
       'kenlm_scores_preceding', 'kenlm_scores_unk', 'kenlm_scores_words',
       'sCounter', 'sentence_index', 'word', 'code', 'input_subject',
       'output_subject'],
      dtype='object')

# Merge With Word Properties

In [144]:
wdfr['word'] = wdfr['bnc_unigram_word']
lexiconch = pd.read_csv('data/lexiconch.csv', index_col=0)
print('Number of words before merging with Lexiconch: '+str(wdfr.shape[0]))
wdfl = wdfr.merge(lexiconch, how='left')
print('Number of words after merging with Lexiconch: '+str(wdfl.shape[0]))
' '.join(set(wdfr.word) - set(wdfl.word))

Number of words before merging with Lexiconch: 27290
Number of words after merging with Lexiconch: 27290


/home/stephan/notebooks/telephone-analysis-public/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9,10,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


''

In [145]:
lexiconch.iloc[0]

word                               roadsweeper
conc_bigram                                  0
conc_conc_m                               4.85
conc_conc_sd                              0.37
conc_unknown                                 1
conc_total                                  27
conc_percent_known                        0.96
conc_subtlex                                 0
conc_dom_pos                                 0
kpm_alternative_spelling          road sweeper
kpm_freq_pm                                NaN
kpm_dom_pos_subtlex                        NaN
kpm_nletters                                11
kpm_nphon                                    8
kpm_nsyll                                    3
kpm_lemma_highest_pos              roadsweeper
kpm_aoa_kup                               9.83
kpm_perc_known                               1
kpm_aoa_kup_lem                           9.83
kpm_perc_known_lem                           1
kpm_aoa_bird_lem                          7.12
kpm_aoa_brist

In [146]:
yarkoni_pld = pd.read_table('data/pld20.txt', header=None)
yarkoni_pld.columns = ['word', 'pld20']

/home/stephan/notebooks/telephone-analysis-public/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [147]:
print('Number of words before merging with Yarkoni PLD: '+str(wdfl.shape[0]))
wdfy = wdfl.merge(yarkoni_pld, how='left')
print('Number of words after merging with Yarkoni PLD: '+str(wdfy.shape[0]))

Number of words before merging with Yarkoni PLD: 27290
Number of words after merging with Yarkoni PLD: 27290


In [148]:
subtlex = pd.read_csv('data/subtlex_augmented.csv')
subtlex['word'] = subtlex.Word
subtlex.iloc[0]

Word                         the
FREQcount                1501908
CDcount                     8388
FREQlow                  1339811
Cdlow                       8388
SUBTLWF                  29449.2
Lg10WF                    6.1766
SUBTLCD                      100
Lg10CD                    3.9237
ipa_                      [ð, ə]
ipa_single                [ð, ə]
ipa_diphthongs            [ð, ə]
unigram_probability    0.0302076
unigram_surprisal        3.49966
word                         the
Name: 0, dtype: object

In [149]:
print('Number of words before merging with Subtlex: '+str(wdfy.shape[0]))
wdfx = wdfy.merge(subtlex[['word','SUBTLCD']], how='left')
print('Number of words after merging with Subtlex: '+str(wdfx.shape[0]))

Number of words before merging with Subtlex: 27290
Number of words after merging with Subtlex: 27290


# Merge in Sentence Level Predictors

In [150]:
all_runs['sentence_index'] = range(all_runs.shape[0])

In [152]:
#wdfs = wdfr.merge(all_runs, on='sentence_index') # !!! temporarily remove additional predictors
wdfs = wdfx.merge(all_runs, on='sentence_index') 

In [153]:
wdfs.loc[wdfs.sentence_index == 1][['bnc_unigram_word',
                                   'user_candidate_transcription',
                                    'gold_candidate_transcription',
                                    'input_subject',
                                    'output_subject'
                                   ]]

,bnc_unigram_word,user_candidate_transcription,gold_candidate_transcription,input_subject,output_subject
10,each,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
11,non,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
12,fiction,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
13,book,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
14,has,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
15,a,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
16,call,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
17,number,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
18,in,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc
19,its,each non fiction book has a call number in its...,each nonfiction book has a call number on its ...,8cf6535ea0ae4addb28f5f90a2b13a7d,edf8f705ab7241a191fa3ae24e381ebc


In [154]:
' '.join(wdfs.columns)

'bnc_unigram_index bnc_unigram_prob bnc_unigram_word bnc_trigram_index bnc_trigram_prob bnc_trigram_word roark_scores_prefix roark_scores_word roark_scores_prefix.1 roark_scores_srprsl roark_scores_SynSp roark_scores_LexSp roark_scores_ambig roark_scores_open roark_scores_rernk roark_scores_toprr roark_scores_stps big_lm_scores_Unnamed: 0 big_lm_scores_prob big_lm_scores_word kenlm_scores_prob kenlm_scores_preceding kenlm_scores_unk kenlm_scores_words sCounter sentence_index word code input_subject output_subject conc_bigram conc_conc_m conc_conc_sd conc_unknown conc_total conc_percent_known conc_subtlex conc_dom_pos kpm_alternative_spelling kpm_freq_pm kpm_dom_pos_subtlex kpm_nletters kpm_nphon kpm_nsyll kpm_lemma_highest_pos kpm_aoa_kup kpm_perc_known kpm_aoa_kup_lem kpm_perc_known_lem kpm_aoa_bird_lem kpm_aoa_bristol_lem kpm_aoa_cort_lem kpm_aoa_schock pic_ipa pic_ipa_ss_array pic_ipa_ss pic_ipa_n pic_ortho pic_ortho_ss_array pic_ortho_ss pic_ortho_n pic_character pic_character_ss_a

# Find Good Example Thread

In [186]:
all_runs.loc[all_runs.thread_id == '0_5']['gold_candidate_transcription']

383                                                 none
384    your teeth begin breaking up the food by chewi...
385    your teeth begin breaking up the food by chewi...
386    your teeth end up breaking up the food by chewing
387    Your teeth end up breaking up the food by chewing
388    your teeth end up breaking up the food by chewing
389    your teeth end up breaking up the food by chew...
390    her teeth ended up breaking to the food back t...
391     her teeth ended up breaking as the food got hard
392           her key ended up breaking off into her car
393              her key ended up breaking in to her car
394               our key ended up breaking into the car
395              Our key ended up breaking into the door
396              Berkie ended up breaking in to the door
397              Our key ended up breaking into the door
398             Our key ended up breaking in to the door
399              Her key ended up breaking into the door
400              her key ended 

In [191]:
all_runs.loc[all_runs.thread_id == '0_5'][['chain','stimulus_id']].head(1)

,chain,stimulus_id
383,0,5


# Save the Output

In [155]:
wdfs.shape

(27290, 170)

In [156]:
wdfs.to_csv('output/wordLevelChanges.csv', index=False)

See R notebook, Error Prediction Model - Logistic Regression